# Submit GW calculation for molecules followed by the IC correction

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
# AiiDAlab imports.
import aiidalab_widgets_base as awb
from aiida import orm, plugins

# Custom imports.
from surfaces_tools.widgets import analyze_structure, computational_resources, inputs

In [ ]:
Cp2kAdsorbedGwIcWorkChain = plugins.WorkflowFactory("nanotech_empa.cp2k.ads_gw_ic")

In [ ]:
# Structure selector.
input_details = inputs.InputDetails()

structure_selector = awb.StructureManagerWidget(
    importers=[
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.StructureUploadWidget(title="Import from computer"),
    ],
    storable=False,
    node_class="StructureData",
)

ipw.dlink((structure_selector, "structure"), (input_details, "structure"))
display(structure_selector)

# Code.
code_input_widget = awb.ComputationalResourcesWidget(
    description="CP2K code:", default_calc_job_plugin="cp2k"
)

In [ ]:
# gw_type
gw_type = ipw.Dropdown(
    options=["gpw_std", "gapw_std", "gapw_hq"],
    value="gpw_std",
    description="Protocol:",
    disabled=False,
)


# geometry mode
geo_mode = ipw.Dropdown(
    options=["ads_geo", "gas_opt"],
    value="ads_geo",
    description="Geo mode:",
    disabled=False,
)

# geometry mode
ads_height = ipw.FloatText(
    value=0.0,
    description="adsorption height in Å:",
    disabled=False,
    style={"description_width": "initial"},
)
# description
description = ipw.Text(
    value="",
    placeholder="Calculation description",
    description="Description:",
    disabled=False,
    style={"description_width": "initial"},
)

# molecule
molecule = ipw.Text(
    value="",
    placeholder="1..2",
    description="Molecule ids:",
    disabled=False,
    style={"description_width": "initial"},
)

# error message
error_msg = ipw.HTML(value="")

# description adsorption height
html_ads_height = ipw.HTML(
    value="""
                <p style="font-weight:400;">If you specify a value >0
                    <font style="font-style:italic;font-weight:400;">(mandatory in case there is no slab)</font> it will override the height extracted from the geometry.
                </p>
                <p>The substrate surface is at <font style="font-style:italic;font-weight:600;">geometric center of the molecule - adsorption height</font></p>
               """
)

In [ ]:
ipw.dlink((code_input_widget, "value"), (input_details, "selected_code"))


def get_builder_gw():
    with output:
        clear_output()
    if not structure_selector.structure_node:
        can_submit, msg = False, "Select a structure first."
    elif not code_input_widget.value:
        can_submit, msg = False, "Select CP2K code."
    else:
        can_submit, msg, parameters = input_details.return_final_dictionary()

    if not can_submit:
        with output:
            print(msg)
            return
    builder = Cp2kAdsorbedGwIcWorkChain.get_builder()
    builder.metadata.label = "CP2K_GWIC"
    builder.metadata.description = description.value
    builder.code = orm.load_code(code_input_widget.value)
    builder.geometry_mode = orm.Str(geo_mode.value)

    # Override automatic adsorption height.
    if ads_height.value > 0.0:
        builder.ads_height = orm.Float(ads_height.value)

    ase_geom = structure_selector.structure

    builder.structure = structure_selector.structure_node
    # builder.magnetization_per_site = orm.List(list=mag_list)
    builder.dft_params = orm.Dict(parameters["dft_params"])
    builder.sys_params = orm.Dict(
        {"molecule_atoms": awb.utils.string_range_to_list(molecule.value)[0]}
    )
    builder.protocol = orm.Str(gw_type.value)

    builder.geometry_mode = orm.Str(geo_mode.value)
    # builder.molecule_atoms = orm.List(awb.utils.string_range_to_list(molecule.value)[0])
    builder.debug = orm.Bool(False)

    builder.options.scf = {
        "max_wallclock_seconds": resources_scf.walltime_seconds,
        "resources": {
            "num_machines": resources_scf.nodes,
            "num_mpiprocs_per_machine": resources_scf.tasks_per_node,
            "num_cores_per_mpiproc": resources_scf.threads_per_task,
        },
    }
    builder.options.gw = {
        "max_wallclock_seconds": resources_gw.walltime_seconds,
        "resources": {
            "num_machines": resources_gw.nodes,
            "num_mpiprocs_per_machine": resources_gw.tasks_per_node,
            "num_cores_per_mpiproc": resources_gw.threads_per_task,
        },
    }
    builder.options.ic = {
        "max_wallclock_seconds": resources_ic.walltime_seconds,
        "resources": {
            "num_machines": resources_ic.nodes,
            "num_mpiprocs_per_machine": resources_ic.tasks_per_node,
            "num_cores_per_mpiproc": resources_ic.threads_per_task,
        },
    }

    return builder

In [ ]:
def after_submission(_=None):
    structure_selector.value = None


btn_submit_gw = awb.SubmitButtonWidget(
    Cp2kAdsorbedGwIcWorkChain, inputs_generator=get_builder_gw
)
btn_submit_gw.btn_submit.disabled = True

btn_submit_gw.on_submitted(after_submission)

In [ ]:
output = ipw.Output()


def extract_details_on_adsorbed_molecule(all_atoms):
    structure_analyzer = analyze_structure.StructureAnalyzer()
    structure_analyzer.structure = all_atoms
    molecule_indices = awb.utils.string_range_to_list(molecule.value)[0]

    if "all_molecules" in structure_analyzer.details:
        if len(molecule_indices) == 0:
            molecule_indices = [
                item
                for sublist in structure_analyzer.details["all_molecules"]
                for item in sublist
            ]

    if len(molecule_indices) > 0:
        molecule.value = awb.utils.list_to_string_range(molecule_indices)
        structure_analyzer.structure = all_atoms[molecule_indices]
        return structure_analyzer.details
    else:
        molecule.value = ""
        return {}


def check_system(_=None):
    btn_submit_gw.btn_submit.disabled = False
    # check system
    only_one_molecule = input_details.details["system_type"] == "SlabXY"
    only_one_molecule = (
        only_one_molecule or input_details.details["system_type"] == "Molecule"
    )
    only_one_molecule = (
        only_one_molecule and len(awb.utils.string_range_to_list(molecule.value)[0]) > 0
    )
    error_msg.value = (
        "<p style='font-weight:bold; color:red;'>Check the molecule indices</p>"
    )
    if only_one_molecule:
        btn_submit_gw.btn_submit.disabled = False
        error_msg.value = ""


def update_all(name=None):

    if name["name"] == "structure":
        input_details.gwic = True
        molecule.value = ""
        error_msg.value = ""
        check_system()
        if structure_selector.structure is not None:
            extract_details_on_adsorbed_molecule(structure_selector.structure)
    with output:
        clear_output()
        to_display = [
            gw_type,
            geo_mode,
            ipw.HBox([ads_height, html_ads_height]),
            molecule,
            input_details,
        ]
        display(ipw.VBox(to_display))


structure_selector.observe(update_all, names="structure")
molecule.observe(check_system, names="value")

In [ ]:
# Resources.
resources_scf = computational_resources.ProcessResourcesWidget()
resources_gw = computational_resources.ProcessResourcesWidget()
resources_ic = computational_resources.ProcessResourcesWidget()

# Resources estimation.
resources_estimation_scf = computational_resources.ResourcesEstimatorWidget(
    calculation_type="dft"
)
resources_estimation_gw = computational_resources.ResourcesEstimatorWidget(
    calculation_type="gw"
)
resources_estimation_ic = computational_resources.ResourcesEstimatorWidget(
    calculation_type="gw_ic"
)

# Link resources widgets.
resources_estimation_scf.link_to_resources_widget(resources_scf)
resources_estimation_gw.link_to_resources_widget(resources_gw)
resources_estimation_ic.link_to_resources_widget(resources_ic)

# Link viewer to resources estimation widgets.
ipw.dlink(
    (structure_selector, "structure"),
    (resources_estimation_scf, "details"),
    transform=extract_details_on_adsorbed_molecule,
)
ipw.dlink(
    (structure_selector, "structure"),
    (resources_estimation_gw, "details"),
    transform=extract_details_on_adsorbed_molecule,
)
ipw.dlink(
    (structure_selector, "structure"),
    (resources_estimation_ic, "details"),
    transform=extract_details_on_adsorbed_molecule,
)

# Link code selector to resources estimation widgets.
_ = ipw.dlink((code_input_widget, "value"), (resources_estimation_scf, "selected_code"))
_ = ipw.dlink((code_input_widget, "value"), (resources_estimation_gw, "selected_code"))
_ = ipw.dlink((code_input_widget, "value"), (resources_estimation_ic, "selected_code"))

# Link UKS
_ = ipw.dlink((input_details, "uks"), (resources_estimation_scf, "uks"))
_ = ipw.dlink((input_details, "uks"), (resources_estimation_gw, "uks"))
_ = ipw.dlink((input_details, "uks"), (resources_estimation_ic, "uks"))

# Link details
_ = ipw.dlink((input_details, "details"), (resources_estimation_scf, "details"))
_ = ipw.dlink((input_details, "details"), (resources_estimation_gw, "details"))
_ = ipw.dlink((input_details, "details"), (resources_estimation_ic, "details"))

# Estimate all resources
estimate_nodes_button = ipw.Button(
    description="Estimate resources", button_style="warning"
)
estimate_nodes_button.on_click(resources_estimation_scf.estimate_resources)
estimate_nodes_button.on_click(resources_estimation_gw.estimate_resources)
estimate_nodes_button.on_click(resources_estimation_ic.estimate_resources)

In [ ]:
display(
    output,
    description,
    error_msg,
    ipw.HBox(
        [
            ipw.VBox(
                [
                    ipw.HTML("DFT resources"),
                    resources_scf,
                ]
            ),
            ipw.VBox(
                [
                    ipw.HTML("GW resources"),
                    resources_gw,
                ]
            ),
            ipw.VBox(
                [
                    ipw.HTML("IC resources"),
                    resources_ic,
                ]
            ),
        ]
    ),
    estimate_nodes_button,
    code_input_widget,
    btn_submit_gw,
)